In [20]:
from copy import deepcopy

import torch

In [21]:
torch.manual_seed(0);

In [22]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [23]:
from senmodel.model.utils import *

In [24]:
class SimpleFCN(nn.Module):
    def __init__(self, input_size=8):
        super(SimpleFCN, self).__init__()
        self.fc1 = nn.Linear(input_size, 4)
        self.fc2 = nn.Linear(4, 4)
        self.fc3 = nn.Linear(2, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x


model = SimpleFCN()

In [25]:
sparse_model = convert_dense_to_sparse_network(model)

In [26]:
linear = nn.Linear(8, 1)

sparse_weight = dense_to_sparse(linear.weight.data)
sparse_bias = dense_to_sparse(linear.bias.data)
just_sparse_linear = ExpandingLinear(sparse_weight, sparse_bias)

sparse_linear = deepcopy(just_sparse_linear)

print(sparse_linear.weight_indices, "\n")
sparse_linear.replace(0, 6, 0)
sparse_linear.replace(0, 7, 0)
print(sparse_linear.weight_indices)
print(sparse_linear.embed_linears[0].weight_indices, "\n")

sparse_linear.replace(0, 8, 1)
sparse_linear.replace(0, 9, 1)
print(sparse_linear.weight_indices)
print(sparse_linear.embed_linears[1].weight_indices, "\n")

sparse_linear.replace(0, 11, 2)
print(sparse_linear.weight_indices)
print(sparse_linear.embed_linears[2].weight_indices, "\n")



tensor([[0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 2, 3, 4, 5, 6, 7]]) 

tensor([[0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 2, 3, 4, 5, 8, 9]])
tensor([[0, 1],
        [6, 7]]) 

tensor([[ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  1,  2,  3,  4,  5, 10, 11]])
tensor([[0, 1],
        [8, 9]]) 

tensor([[ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  1,  2,  3,  4,  5, 10, 11]])
tensor([[ 0],
        [11]]) 



In [27]:
from senmodel.metrics.nonlinearity_metrics import *


In [28]:
criterion = nn.MSELoss()

metrics = [
    GradientMeanEdgeMetric(criterion),
    PerturbationSensitivityEdgeMetric(criterion),
]

In [29]:
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_regression

X, y = make_regression(n_samples=1000, n_features=8, n_informative=8, random_state=42)
X = torch.from_numpy(X).float()
y = torch.from_numpy(y).float()

dataset = list(zip(X, y))
train_dataset, test_dataset = train_test_split(dataset, test_size=0.2, random_state=42)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [30]:

from senmodel.metrics.edge_finder import *

ef = EdgeFinder(metrics[1], test_loader, device=device)

In [31]:
print("values:", ef.calculate_edge_metric_for_dataloader(sparse_linear))
chosen_edges = ef.choose_edges_top_k(sparse_linear, 4)
print("choose:", chosen_edges)

chosen_edges = ef.choose_edges_top_percent(sparse_linear, 0.5)
print("choose:", chosen_edges)

values: tensor([0.8725, 0.8654, 0.8417, 0.7807, 0.7904, 0.8353, 0.0000, 0.0966])
choose: tensor([[0, 0, 0, 0],
        [0, 1, 2, 5]])
choose: tensor([[0, 0, 0, 0],
        [0, 1, 2, 5]])


In [32]:
sparse_linear.weight_indices

tensor([[ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  1,  2,  3,  4,  5, 10, 11]])

In [33]:
sparse_linear.replace_many(*chosen_edges, 3)

In [34]:
sparse_linear.weight_indices

tensor([[ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 3,  4, 10, 11, 12, 13, 14, 15]])

In [35]:
print("values:", ef.calculate_edge_metric_for_dataloader(sparse_linear))
chosen_edges = ef.choose_edges_top_k(sparse_linear, 5)
print("choose:", chosen_edges)

values: tensor([0.9164, 0.9309, 0.0096, 0.1216, 0.0516, 0.0013, 0.3715, 0.0888])
choose: tensor([[ 0,  0,  0,  0,  0],
        [ 4,  3, 14, 11, 15]])


In [36]:
sparse_linear.replace_many(*chosen_edges, 4)

In [37]:
sparse_linear.weight_indices

tensor([[ 0,  0,  0,  0,  0,  0,  0,  0],
        [10, 12, 13, 16, 17, 18, 19, 20]])

In [38]:
print("values:", ef.calculate_edge_metric_for_dataloader(sparse_linear))
chosen_edges = ef.choose_edges_top_k(sparse_linear, 4)
print("choose:", chosen_edges)

values: tensor([5.8880e-02, 1.5915e-01, 3.9312e-02, 2.4353e-02, 1.0000e+00, 3.3895e-01,
        3.5728e-01, 1.6178e-04])
choose: tensor([[ 0,  0,  0,  0],
        [17, 19, 18, 12]])
